In [29]:
import csv
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM, Dropout,Embedding, Bidirectional
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('arabic'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
vocab_size = 5000
embedding_dim = 64
max_length = 200
oov_tok = '<OOV>' #  Out of Vocabulary
training_portion = 0.8

In [31]:
articles = []
labels = []

with open("Data/arabic_main.csv", 'r',encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labels.append(row[1])
        article = row[0]
        for word in STOPWORDS:
            token = ' ' + word + ' '
            article = article.replace(token, ' ')
            article = article.replace(' ', ' ')
        articles.append(article)

In [32]:
len(labels),len(articles)

(8026, 8026)

In [33]:
print(labels[0])

schedule


In [34]:
train_size = int(len(articles) * training_portion)

train_articles = articles[0: train_size]
train_labels = labels[0: train_size]

validation_articles = articles[train_size:]
validation_labels = labels[train_size:]

In [35]:
print('train_articles' ,len(train_articles))
print('train_labels', len(train_labels))
print('validation_articles', len(validation_articles))
print('validation_labels', len(validation_labels))

train_articles 6420
train_labels 6420
validation_articles 1606
validation_labels 1606


In [36]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_articles)
word_index = tokenizer.word_index

In [37]:
import pickle
filename = f'../../utils/tokenizer.sav'
pickle.dump(tokenizer, open(filename, 'wb'))

In [38]:
word_index

{'<OOV>': 1,
 'هل': 2,
 'ما': 3,
 'الطقس': 4,
 'الساعة': 5,
 '،': 6,
 'يوم': 7,
 'الخاص': 8,
 'سيكون': 9,
 'التقويم': 10,
 'موعد': 11,
 'اليوم': 12,
 'يمكنك': 13,
 'مساءً': 14,
 'كيف': 15,
 'الأسبوع': 16,
 'التحقق': 17,
 'وقت': 18,
 'ستكون': 19,
 'فضلك': 20,
 'من': 21,
 'أريد': 22,
 'الحدث': 23,
 'توقعات': 24,
 'الجو': 25,
 'بي': 26,
 '30': 27,
 'ماذا': 28,
 'لدي': 29,
 'طبيب': 30,
 'حدد': 31,
 'موعدًا': 32,
 'يمكن': 33,
 'إضافة': 34,
 'مارس؟': 35,
 'أود': 36,
 'تاريخ': 37,
 'أحتاج': 38,
 'مارس': 39,
 'تخبرني': 40,
 'صباحًا': 41,
 'أعرف': 42,
 'قل': 43,
 'متى': 44,
 'الأحداث': 45,
 '3': 46,
 '4': 47,
 'يرجى': 48,
 'اليوم؟': 49,
 'القادم': 50,
 '5': 51,
 'الشهر': 52,
 'نعم': 53,
 'درجة': 54,
 'ليوم': 55,
 'كم': 56,
 'الظهر': 57,
 'في': 58,
 'المقبل': 59,
 'شكرا': 60,
 'أخبرني': 61,
 'الحياة': 62,
 '11': 63,
 'أكون': 64,
 'ووقت': 65,
 'الثاني': 66,
 'يكون': 67,
 'تحقق': 68,
 'عندما': 69,
 'منبه': 70,
 '10': 71,
 'يمكنني': 72,
 '15': 73,
 'يجب': 74,
 'بي؟': 75,
 'تذكيرًا': 76,
 '2': 77,
 

In [39]:
tokenizer.texts_to_sequences(['what will be the weather tommorrow morning ?'])

[[1, 1, 1, 1513, 1, 1, 1]]

In [40]:
train_sequences = tokenizer.texts_to_sequences(train_articles)
len(train_sequences)

6420

In [41]:
train_sequences[0]

[3, 37, 97, 612, 613, 306]

In [42]:
train_padded = pad_sequences(train_sequences, maxlen=max_length,)

In [43]:
len(train_padded),len(train_padded[0])

(6420, 200)

In [44]:
validation_sequences = tokenizer.texts_to_sequences(validation_articles)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length)

In [45]:
set(labels)

{'general', 'greeting', 'recommendation', 'schedule', 'thank', 'weather'}

In [46]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels))

In [47]:
validation_label_seq

array([[3],
       [3],
       [1],
       ...,
       [4],
       [2],
       [3]])

In [48]:
# train_labels[744], training_label_seq[744]

In [49]:
model = Sequential()
model.add(Embedding(vocab_size,embedding_dim ))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(embedding_dim)))
model.add(Dense(7,activation='softmax' ))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 64)          320000    
                                                                 
 dropout_1 (Dropout)         (None, None, 64)          0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              66048     
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 7)                 903       
                                                                 
Total params: 386,951
Trainable params: 386,951
Non-trainable params: 0
_________________________________________________________________


In [50]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(loss='sparse_categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

C:\Users\dell\AppData\Roaming\Python\Python310\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [51]:
num_epochs = 10
history = model.fit(train_padded, training_label_seq, epochs=num_epochs, validation_data=(validation_padded, validation_label_seq), verbose=2)

Epoch 1/10
201/201 - 32s - loss: 0.9014 - accuracy: 0.6841 - val_loss: 0.3860 - val_accuracy: 0.8674 - 32s/epoch - 158ms/step
Epoch 2/10
201/201 - 30s - loss: 0.2474 - accuracy: 0.9276 - val_loss: 0.2186 - val_accuracy: 0.9390 - 30s/epoch - 148ms/step
Epoch 3/10
201/201 - 27s - loss: 0.1214 - accuracy: 0.9678 - val_loss: 0.1607 - val_accuracy: 0.9533 - 27s/epoch - 136ms/step
Epoch 4/10
201/201 - 27s - loss: 0.0769 - accuracy: 0.9771 - val_loss: 0.1633 - val_accuracy: 0.9508 - 27s/epoch - 135ms/step
Epoch 5/10
201/201 - 31s - loss: 0.0546 - accuracy: 0.9835 - val_loss: 0.1487 - val_accuracy: 0.9583 - 31s/epoch - 152ms/step
Epoch 6/10
201/201 - 30s - loss: 0.0412 - accuracy: 0.9874 - val_loss: 0.1463 - val_accuracy: 0.9570 - 30s/epoch - 151ms/step
Epoch 7/10
201/201 - 30s - loss: 0.0324 - accuracy: 0.9894 - val_loss: 0.1593 - val_accuracy: 0.9552 - 30s/epoch - 149ms/step
Epoch 8/10
201/201 - 31s - loss: 0.0267 - accuracy: 0.9919 - val_loss: 0.1772 - val_accuracy: 0.9483 - 31s/epoch - 156

In [59]:
txt_list = ["ما حال الطقس اليوم ؟","احجز لى معاد اليوم الساعة 5","كان بوم جميل جدا"," اقترح فيلم جديد لاشاهده"]
labels = ['schedule','weather','general','recommendation','sports','greeting','thank']
answer = list()
for txt in txt_list:
    seq = tokenizer.texts_to_sequences([txt])
    padded = pad_sequences(seq, maxlen=max_length)
    pred = model.predict(padded)
    try:
        label = labels[np.argmax(pred)-1] 
        answer.append(label)
    except:
        answer.append('general')
print(answer)

1/1 [==============================] - 0s 31ms/step
['weather', 'schedule', 'general', 'recommendation']


In [60]:
model.save("models")

INFO:tensorflow:Assets written to: models\assets


INFO:tensorflow:Assets written to: models\assets


In [61]:
from tensorflow import keras

m = keras.models.load_model("models")

In [62]:
answer = list()
for txt in txt_list:
    seq = tokenizer.texts_to_sequences([txt])
    padded = pad_sequences(seq, maxlen=max_length)
    pred = m.predict(padded)
    try:
        label = labels[np.argmax(pred)-1] 
        answer.append(label)
    except:
        answer.append('general')
print(answer)

1/1 [==============================] - 0s 137ms/step
['weather', 'schedule', 'general', 'recommendation']
